# Agilepy science API tutorial - VELA Region analysis

* The purpose of this notebook is to show how to use the Agilepy scientific API. 


## The complete API documentation can be found [at this link](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html).

## Importing the library

In [1]:
from agilepy.api.AGAnalysis import AGAnalysis

In [ ]:
# Interactive plot
%matplotlib widget

## Creating a configuration file

In [2]:
confFilePath = "$HOME/agilepy_conf.yaml"

In [3]:
AGAnalysis.getConfiguration(
    evtfile="$AGILE/agilepy-test-data/test_dataset_6.0/EVT/EVT.index",
    logfile="$AGILE/agilepy-test-data/test_dataset_6.0/LOG/LOG.index",
    confFilePath = confFilePath,
    userName = "user-xxx",
    sourceName = "vela-xxx",
    tmin = 433857532,
    tmax = 434289532,
    timetype = "TT",
    glon = 263.55,
    glat = -2.78,
    outputDir = "$HOME/agilepy_analysis",
    verboselvl = 0
)

## Initialization of the class

In [4]:
ag = AGAnalysis(confFilePath)

## API: configuration file 

In [5]:
#print all options of the configuration file
ag.printOptions()

{ 'ap': {'radius': 3, 'timeslot': 3600},
  'input': { 'evtfile': '/Users/bulgarelli/opt/anaconda3/envs/agilepyenv/agiletools/agilepy-test-data/test_dataset_6.0/EVT/EVT.index',
             'logfile': '/Users/bulgarelli/opt/anaconda3/envs/agilepyenv/agiletools/agilepy-test-data/test_dataset_6.0/LOG/LOG.index'},
  'maps': { 'binsize': 0.25,
            'energybins': [[100, 10000]],
            'expstep': 4.0,
            'fovbinnumber': 1,
            'mapsize': 40,
            'offaxisangle': 30,
            'proj': 'ARC',
            'projtype': 'WCS',
            'spectralindex': 2.1,
            'timestep': 160,
            'useEDPmatrixforEXP': False},
  'mle': { 'contourpoints': 40,
           'edpcorrection': 0.75,
           'expratio_maxthr': 15,
           'expratio_minthr': 0,
           'expratio_size': 10,
           'expratioevaluation': True,
           'fluxcorrection': 0,
           'integratortype': 1,
           'loccl': 5.99147,
           'mindefaulttolerance': 0.01,

In [6]:
#print a single section of the yaml configuration file
ag.printOptions("maps")

{ 'binsize': 0.25,
  'energybins': [[100, 10000]],
  'expstep': 4.0,
  'fovbinnumber': 1,
  'mapsize': 40,
  'offaxisangle': 30,
  'proj': 'ARC',
  'projtype': 'WCS',
  'spectralindex': 2.1,
  'timestep': 160,
  'useEDPmatrixforEXP': False}


In [7]:
#print a single option
ag.getOption("energybins")

[[100, 10000]]

In [8]:
#set some options
ag.setOptions(energybins=[[100,300], [300, 1000]], mapsize=50, binsize=0.4)

In [9]:
ag.printOptions("maps")

{ 'binsize': 0.4,
  'energybins': [[100, 300], [300, 1000]],
  'expstep': 4.0,
  'fovbinnumber': 1,
  'mapsize': 50,
  'offaxisangle': 30,
  'proj': 'ARC',
  'projtype': 'WCS',
  'spectralindex': 2.1,
  'timestep': 160,
  'useEDPmatrixforEXP': False}


## API: interacting with the Sources Library

### Loading the AGILE's second catalogue of gamma-ray sources

In [10]:
sources = ag.loadSourcesFromCatalog("2AGL", rangeDist = (0, 25))

In [11]:
type(sources[0])

agilepy.core.SourceModel.Source

In [12]:
for s in sources:
    print(s)


-----------------------------------------------------------
Source name: 2AGLJ1045-5954 (PointSource)
  * Position:
	- start_pos: (287.697, -0.788886)
	- dist from (l,b): 24.2159
  * Spectrum: (PowerLaw)
	- flux: 1.80893e-07
	- index: 2.0414
  * Free params: none
-----------------------------------------------------------

-----------------------------------------------------------
Source name: 2AGLJ1048-5836 (PointSource)
  * Position:
	- start_pos: (287.422, 0.423015)
	- dist from (l,b): 24.0778
  * Spectrum: (PowerLaw)
	- flux: 1.54772e-07
	- index: 2.00039
  * Free params: none
-----------------------------------------------------------

-----------------------------------------------------------
Source name: 2AGLJ1045-5735 (PointSource)
  * Position:
	- start_pos: (286.672, 1.23211)
	- dist from (l,b): 23.4607
  * Spectrum: (PowerLaw)
	- flux: 1.64949e-07
	- index: 2.25973
  * Free params: none
-----------------------------------------------------------

-------------------------

### Alternative solution: loading the sources from a custom AGILE .txt file
The sources that are already present in the Sources Library will not be loaded twice.

In [ ]:
sources_hypotesis = """
969.539e-08 263.585 -2.84083 1.71345 0 2 2AGLJ0835-4514 0.0 2 3913.06 1.34774 0.5 5 20 10000 0 100
10e-08 273.0 -1.1 2.1 1 2 NEWOURCE 0.0 0 0 0 0.5 5 20 10000 0 100
"""
with open("./sources_hyp.txt", "w") as sf:
    sf.write(sources_hypotesis)

In [ ]:
#sources = ag.loadSourcesFromFile("./sources_hyp.txt", rangeDist = (0, 25))

### Selecting the sources
The selection criteria can be expressed using the following Source class’s parameters:

* name: the unique code identifying the source.
* dist: the distance of the source from the center of the maps.
* flux: the flux value.
* sqrtTS: the radix square of the ts.

In [13]:
selectedSources = ag.selectSources('flux > 0')

In [14]:
len(selectedSources)

9

In [15]:
selectedSources = ag.selectSources('name == "2AGLJ0835-4514"')

In [16]:
len(selectedSources)
for s in selectedSources:
    print(s)


-----------------------------------------------------------
Source name: 2AGLJ0835-4514 (PointSource)
  * Position:
	- start_pos: (263.585, -2.84083)
	- dist from (l,b): 0.0702
  * Spectrum: (PLSuperExpCutoff)
	- flux: 9.69539e-06
	- index1: 1.71345
	- cutoffEnergy: 3913.06
	- index2: 1.34774
  * Free params: none
-----------------------------------------------------------


In [17]:
selectedSources = ag.selectSources("flux > 0 AND dist <= 2")

In [ ]:
len(selectedSources)
for s in selectedSources:
    print(s)

### Free/Fix source parameters
You can fix or free the following parametes:

* flux
* index
* index1
* index2
* cutoffEnergy
* pivotEnergy
* curvature
* index2
* pos for the position of the source

In [18]:
_sources = ag.freeSources('name == "2AGLJ0835-4514"', "flux", True, show=True)

In [ ]:
for s in _sources:
    print(s)

In [ ]:
_sources = ag.freeSources('name == "2AGLJ0835-4514"', "index1", True)

In [ ]:
for s in _sources:
    print(s)

In [ ]:
_sources = ag.freeSources("flux > 0 AND dist <= 2", "flux", True)

In [ ]:
#len is 0 because the flux is already free
len(_sources)

In [ ]:
for s in _sources:
    print(s)

Resetting the changes:

In [ ]:
_sources = ag.freeSources('name == "2AGLJ0835-4514"', "flux", False)

In [ ]:
ag.freeSources('name == "2AGLJ0835-4514"', "index1", False)

In [ ]:
ag.freeSources("flux > 0 AND dist <= 2", "flux", False)

### Deleting sources

In [ ]:
deleted = ag.deleteSources('name == "2AGLJ1048-5836"').pop()
print(deleted)

In [ ]:
len(ag.getSources())

### Adding a source

Passing a dictionary:

In [ ]:
newSourceDict = {
    "glon" : 273.0,
    "glat":  -1.1,
    "spectrumType" : "PowerLaw",
    "flux": 10e-08,
    "index": 2.1
}

if ag.addSource("NEWSOURCE", newSourceDict):
    print("Source loaded")

In [ ]:
len(ag.getSources())

Passing a Source object:

In [ ]:
type(deleted)

In [ ]:
if ag.addSource("NEWSOURCE", deleted):
    print("Source loaded")
else:
    print("Source is already present in the SourcesLibrary")

In [ ]:
deletedSources = ag.deleteSources('name == "NEWSOURCE"')

In [ ]:
len(ag.getSources())

## Update the value of a spectrum parameter of a source

In [ ]:
sources = ag.selectSources('name == "2AGLJ0835-4514"')
source = sources.pop()
source.spectrum.set("index2", 1.34774)
print(source)

## Fix the hypothesis for the analysis

In [ ]:
# This method returns the selected sources affected by the change
sources = ag.freeSources('name == "2AGLJ0835-4514"', "flux", True, show=True)

In [ ]:
#final source hypothesis for the analysis
selectedSources = ag.selectSources('flux > 0')
print(len(selectedSources))
for s in selectedSources:
    print(s)

## API: generate maps and perform maximum likelihood estimator analysis

### Generating maps

In [ ]:
ag.printOptions("maps")

In [ ]:
maplistfile = ag.generateMaps()

### Displaying maps

#### Single figure - multiple subplots

In [ ]:
ag.displayCtsSkyMaps(smooth=2, regFilePath="$AGILE/catalogs/3EG_1.reg", catalogRegions="2AGL", catalogRegionsColor="yellow")

#### Multiple figures - one subplot per figure

In [ ]:
ag.displayExpSkyMaps(singleMode = False)

In [ ]:
ag.displayGasSkyMaps()

### Saving maps

In [ ]:
ag.displayCtsSkyMaps(smooth=3, saveImage=True)

### Estimation of background coefficients
You can use the [calcBkg()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.calcBkg) method to estimate the background coefficients.

In [ ]:
print("isocoeff: {} galcoeff: {}".format(ag.getOption("isocoeff"), ag.getOption("galcoeff")))

In [ ]:
isoBkg, galBkg, maplistfile = ag.calcBkg("2AGLJ0835-4514", galcoeff = [0.7, 0.7], pastTimeWindow = 0)

In [ ]:
# calcBkg with pastTimeWindow = 14
# ag.setOptions(galcoeff=[0.469176, 0.699403, 0.199108, 0.533379], isocoeff=[4.27314, 0.976644, 8.12386, 1.24627])

In [ ]:
print("isocoeff: {} galcoeff: {}".format(ag.getOption("isocoeff"), ag.getOption("galcoeff")))

### Maximum likelyhood analysis

In [ ]:
sourceFiles = ag.mle()

In [ ]:
#print results
sources = ag.selectSources("sqrtTS > 0")
print(len(sources))
for s in sources:
    print(s)

In [ ]:
#free also position
ag.freeSources('name == "2AGLJ0835-4514"', "pos", True, show=True)

In [ ]:
ag.mle()

In [ ]:
#print results
sources = ag.selectSources("sqrtTS > 0")
print(len(sources))
for s in sources:
    print(s)

## Light curve

In [ ]:
#fix the position and keep only flux free
ag.freeSources('name == "2AGLJ0835-4514"', "pos", False, show=True)

In [ ]:
lightCurveData = ag.lightCurveMLE("2AGLJ0835-4514", binsize=86400)

In [ ]:
! cat $lightCurveData

In [ ]:
ag.displayLightCurve("mle")

## Aperture Photometry

In [ ]:
ap_file, _ = ag.aperturePhotometry()

In [ ]:
! cat $ap_file

In [ ]:
ag.displayLightCurve("ap")

## Cleaning up

In [ ]:
ag.deleteAnalysisDir()